In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import pickle as pk
import resources.RES as res
from sklearn.preprocessing import minmax_scale as sk_minmax

In [2]:
# retrieve context
context_path = os.path.relpath("../input/testcase_context.obj") 
context_mesh = tm.load(context_path)

In [3]:
# retrieve lattice
env_lat_path = os.path.relpath("../data/testcase_lattice.csv")
envelope_lattice = tg.lattice_from_csv(env_lat_path)

In [4]:
envelope_lattice.centroids.shape, envelope_lattice.size,envelope_lattice.centroids.shape

((315, 3), 315, (315, 3))

In [5]:
# retrieve skydome
skydome = pk.load(open("../data/T2_skydome.pk", "rb"))

In [6]:
# transformation matrix
def transform_mat(value):
    mat = np.identity(4)
    mat[:3,-1] = np.array(value)
    return mat

In [7]:
# voxel cuboid meshes
vox_cuboids = [tm.creation.box(envelope_lattice.unit, transform=transform_mat(ct)) for ct in envelope_lattice.centroids]

# number of faces per voxel
f_numb = len(vox_cuboids[0].faces)

# combine voxels into one mesh
combined_voxels = tm.util.concatenate(vox_cuboids)

In [8]:
# combine voxels and context into one mesh
combined_meshes = tm.util.concatenate(combined_voxels, context_mesh)

# shoot towards the skydome points from all of the voxels
DL_ray_ctr = np.tile(skydome, [len(envelope_lattice.centroids),1]) # daylighting ray for each centroid
DL_ctr_ray = np.tile(envelope_lattice.centroids, [1, len(skydome)]).reshape(-1, 3) # daylighting centroid for each ray

In [9]:
# intersection of rays from voxel centroids to sky patch objects with all voxel faces
face_id, ray_id = combined_meshes.ray.intersects_id(ray_origins=DL_ctr_ray, ray_directions=DL_ray_ctr, multiple_hits=True)

In [10]:
G1, U1 = res.construct_graph(skydome, face_id, ray_id, envelope_lattice, f_numb)

In [11]:
G1.shape, U1.shape

((315, 315, 305), (315, 305))

In [12]:
# save interedependencies for later use
pk.dump(G1, open("../data/T3_SkyG1.pk", "wb"))

# save obstructions for later use
pk.dump(U1, open("../data/T3_SkyU1.pk", "wb"))